In [ ]:
# !conda update conda -y
# !conda update conda -y
# !conda install psycopg2 -y

In [1]:
import psycopg2
from psycopg2 import errorcodes, errors
import os, sys, getpass

In [2]:
getpass.getuser()

'seanreed1'

In [3]:
conn = psycopg2.connect(database="jupyterdb", user=getpass.getuser())


In [4]:
cur = conn.cursor()
cur.execute("SELECT 1,2,3;")
conn.commit()
cur.fetchall()

[(1, 2, 3)]

Note: If you don't want to have **conn.commit()** everywhere in your code, set

conn.autocommit = True

In [5]:
# let's cleanup
cur.close(), conn.close()

In [6]:
# what if we get the database name wrong?
conn = psycopg2.connect(database="doesnotexist", user=getpass.getuser())

OperationalError: FATAL:  database "doesnotexist" does not exist


In [ ]:
## let's catch the Operational Error for db does not exist now

https://www-freecodecamp-org.cdn.ampproject.org/c/s/www.freecodecamp.org/news/connect-python-with-sql/amp/

In [ ]:
# # define a function that handles and parses psycopg2 exceptions
# MIGHT NOT NEED THIS ANYMORE
# def print_psycopg2_exception(err):
#     # get details about the exception
#     err_type, err_obj, traceback = sys.exc_info()

#     # get the line number when exception occured
#     line_num = traceback.tb_lineno
#     # print the connect() error
#     print (f"\npsycopg2 ERROR: {err}, on line number:{line_num}\n")
#     print (f"psycopg2 traceback: {traceback}\ntype: {err_type}\n")

#     # psycopg2 extensions.Diagnostics object attribute
#     print (f"\nextensions.Diagnostics: {err.diag}")

#     # print the pgcode and pgerror exceptions
#     print (f"pgerror: {err.pgerror}\n")
#     print (f"pgcode: {err.pgcode} \n")

In [7]:
def create_server_connection(database, user):
    conn = None
    try:
        conn = psycopg2.connect(
            database=database,
            user=user,
        )
        print(f"Connected to Database {database} successfully")
        
    except Exception as e:
        print(e)

    return conn

In [8]:
conn = create_server_connection(database="doesnotexist", user=getpass.getuser())

FATAL:  database "doesnotexist" does not exist



In [9]:
#let's make sure the create_server_connection can still connect normally

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = conn.cursor()
cur.execute("SELECT 1,2,3,4,5;")
conn.commit()
cur.fetchall()

Connected to Database jupyterdb successfully


[(1, 2, 3, 4, 5)]

In [10]:
# let's cleanup

cur.close(), conn.close()

(None, None)

In [11]:
# Let's check error handling on SQL statements

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())

#the context manager will close the cursor when it exists. 
# GOTCHA: YOU MUST CLOSE THE CONNECTION EXPLICITLY (context manager does not close the conn)

with conn.cursor() as cursor:
    try:
        cursor.execute("SELECT * FROM NOTABLE;")
        conn.commit()
    except Exception as e:
        print(e)

conn.close()


Connected to Database jupyterdb successfully
relation "notable" does not exist
LINE 1: SELECT * FROM NOTABLE;
                      ^



In [ ]:
# let's write a create/drop database function 

In [37]:
def create_or_drop_database(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    with conn.cursor() as cur:
        try:
            cur.execute(query)
            print("Database created/dropped successfully.")
        except Exception as e:
            print(e)
        finally:
            #be a good citizen and put stuff back the way you found it
            conn.autocommit = autocommit 

In [36]:
q = "CREATE DATABASE school4;"

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
create_or_drop_database(conn, q)
conn.close()

Connected to Database jupyterdb successfully
Database created/dropped successfully.


In [38]:
q = "DROP DATABASE school4;"

conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
create_or_drop_database(conn, q)
conn.close()

Connected to Database jupyterdb successfully
Database created/dropped successfully.


In [1]:
# let's write an execute query function
# we want our function to return a cursor with the results

def execute_query(conn, query):
    autocommit = conn.autocommit #preserve the value of autocommit
    conn.autocommit = True #MUST BE set to True since Postgresql cannot create a database in a transaction
    cur = conn.cursor()
    try:
        cur.execute(query)
        print("Query ran successfully.")
    except Exception as e:
        print(e)
    finally:
        #be a good citizen and put stuff back the way you found it
        conn.autocommit = True 
    return cur


In [44]:
q = "SELECT 1,2,3,4,5"
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = execute_query(conn, q)
print(cur.fetchall())
cur.close()
conn.close()

Connected to Database jupyterdb successfully
Query ran successfully.
[(1, 2, 3, 4, 5)]


In [48]:
q = "SELECT 1,2,3,4,5 FROM doesnotexist"
conn = create_server_connection(database="jupyterdb", user=getpass.getuser())
cur = execute_query(conn, q)
cur.close()
conn.close()

Connected to Database jupyterdb successfully
relation "doesnotexist" does not exist
LINE 1: SELECT 1,2,3,4,5 FROM doesnotexist
                              ^

